In [1]:
import pandas as pd

from data import Dataset as dataset

videos = dataset.getVideos()
comments = dataset.getAllComments()

In [2]:
videos

,kind,videoId,publishedAt,channelId,title,description,tags,defaultLanguage,defaultAudioLanguage,contentDuration,viewCount,likeCount,favouriteCount,commentCount,topicCategories
0,youtube#video,85806,2024-01-15 00:59:29+00:00,33807,Unlocking the Benefits of Face Masks for Skin ...,NaN,NaN,en-US,en-US,PT9S,72.0,0.0,0.0,0.0,"['https://en.wikipedia.org/wiki/Health', 'http..."
1,youtube#video,30556,2023-10-27 19:32:16+00:00,46650,Get ready for the Magic💚💜🤍💝✨ #hydration #glowi...,NaN,NaN,NaN,NaN,PT45S,257.0,7.0,0.0,0.0,['https://en.wikipedia.org/wiki/Lifestyle_(soc...
2,youtube#video,51771,2024-09-28 01:23:22+00:00,14346,#trending #makeup #beautymakeup #yslbeauty #lu...,NaN,NaN,NaN,en-US,PT19S,164.0,4.0,0.0,2.0,['https://en.wikipedia.org/wiki/Lifestyle_(soc...
3,youtube#video,45298,2023-07-13 15:19:28+00:00,50139,#shortvedio #balayage,NaN,NaN,NaN,NaN,PT14S,1207.0,20.0,0.0,0.0,['https://en.wikipedia.org/wiki/Lifestyle_(soc...
4,youtube#video,43611,2023-04-29 18:47:37+00:00,8143,Full Face of Merit Beauty 🤎 featuring new Flus...,NaN,NaN,NaN,en,PT56S,8647.0,268.0,0.0,7.0,['https://en.wikipedia.org/wiki/Lifestyle_(soc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92754,youtube#video,26001,2023-12-28 21:04:42+00:00,35305,New Year Styling! | #Beachwaver Co.,Connect with us and keep up with our brand! Pl...,"['Beachwaver', 'DreamBigMakeWaves', 'Hair Care...",NaN,en,PT30S,1485.0,29.0,0.0,0.0,"['https://en.wikipedia.org/wiki/Fashion', 'htt..."
92755,youtube#video,34584,2021-04-15 12:08:32+00:00,6127,DIY glow serum / in urdu and hindi,#shortvideo#\n#youtubeshortvideo#\n#glowserum#,NaN,NaN,NaN,PT31S,24.0,4.0,0.0,0.0,"['https://en.wikipedia.org/wiki/Hobby', 'https..."
92756,youtube#video,21075,2023-04-27 04:07:06+00:00,7271,How Women at 50 Stay Young❓#wrinkles #senescen...,@Jeffree,NaN,NaN,NaN,PT12S,1422.0,10.0,0.0,3.0,"['https://en.wikipedia.org/wiki/Health', 'http..."
92757,youtube#video,44523,2025-04-27 11:16:59+00:00,42969,makeup try #face primer#concealer#faondation,NaN,NaN,en-IN,hi,PT22S,75.0,1.0,0.0,0.0,['https://en.wikipedia.org/wiki/Lifestyle_(soc...


In [3]:
comments.describe()

,commentId,channelId,videoId,authorId,parentCommentId,likeCount
count,4.725012e+06,4.725012e+06,4.725012e+06,4.725012e+06,5.161350e+05,4.725012e+06
mean,2.362507e+06,2.677102e+04,4.699673e+04,1.820729e+06,2.623663e+06,1.012744e+01
std,1.363995e+06,1.503666e+04,2.593627e+04,1.053868e+06,1.215428e+06,5.444689e+02
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,5.161510e+05,0.000000e+00
25%,1.181254e+06,1.449200e+04,2.597800e+04,9.060968e+05,1.569545e+06,0.000000e+00
50%,2.362506e+06,2.542500e+04,4.709600e+04,1.813632e+06,2.625877e+06,0.000000e+00
75%,3.543761e+06,4.061800e+04,6.944500e+04,2.731818e+06,3.677645e+06,0.000000e+00
max,4.725015e+06,5.367700e+04,9.285400e+04,3.659440e+06,4.725006e+06,4.561420e+05
